In [240]:
# PyTorch
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.utils.data.dataset import random_split
import torch.utils.data as data
import torchvision.transforms as transforms

# SciKit
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
# Python
import pandas as pd
import numpy as np
import time

# Graphing
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline


# utils
from utils import *

# Device (GPU or CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [241]:
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)

In [242]:
# ML Parameters
lr = 1e-3
min_lr = 1e-5
epochs = 100
batch_size = 512

# Data Parameters
data_seq_len = 100
data_n_features = 1
data_embedding_dim = 64

In [243]:
#load train data
df_sensorA_normal = pd.read_csv('data/sensor_A_normal.csv')
df_sensorB_normal = pd.read_csv('data/sensor_B_normal.csv')
df_sensorC_normal = pd.read_csv('data/sensor_C_normal.csv')
df_sensorD_normal = pd.read_csv('data/sensor_D_normal.csv')
df_sensorE_normal = pd.read_csv('data/sensor_E_normal.csv')
print(len(df_sensorA_normal))
df_train = [df_sensorA_normal, df_sensorB_normal, df_sensorC_normal, df_sensorD_normal, df_sensorE_normal]

# load val data
df_sensorA_public = pd.read_csv('data/sensor_A_public.csv')
df_sensorB_public = pd.read_csv('data/sensor_B_public.csv')
df_sensorC_public = pd.read_csv('data/sensor_C_public.csv')
df_sensorD_public = pd.read_csv('data/sensor_D_public.csv')
df_sensorE_public = pd.read_csv('data/sensor_E_public.csv')
print(len(df_sensorA_public))
df_test = [df_sensorA_public, df_sensorB_public, df_sensorC_public, df_sensorD_public, df_sensorE_public]

# load test data
df_sensorA_private = pd.read_csv('data/sensor_A_private.csv')
df_sensorB_private = pd.read_csv('data/sensor_B_private.csv')
df_sensorC_private = pd.read_csv('data/sensor_C_private.csv')
df_sensorD_private = pd.read_csv('data/sensor_D_private.csv')
df_sensorE_private = pd.read_csv('data/sensor_E_private.csv')
print(len(df_sensorA_private))
df_private = [df_sensorA_private, df_sensorB_private, df_sensorC_private, df_sensorD_private, df_sensorE_private]

2875
4000
4000


In [244]:
for df in df_test:
    df.drop(df.index[-1], inplace=True)

In [246]:
class SensorDataset(data.Dataset):
    """
        Support class for the loading and batching of sequences of samples

        Args:
            dataset (Tensor): Tensor containing all the samples
            sequence_length (int): length of the analyzed sequence by the LSTM
            transforms (object torchvision.transform): Pytorch's transforms used to process the data
    """
    ##  Constructor
    def __init__(self, df, seq_len=1, transform=None):
        self.dataset = df
        self.seq_len = seq_len
        self.transforms = transform

    ##  Override total dataset's length getter
    def __len__(self):
        return self.dataset.__len__()

    ##  Override single items' getter
    def __getitem__(self, idx):
        if idx + self.seq_len > len(self.dataset):
            if self.transforms is not None:    
                item = torch.zeros(self.seq_len, self.dataset[0].__len__())
                item[:self.__len__()-idx] = self.transforms(self.dataset[idx:])
                return item, item
            else:
                item = []
                item[:self.__len__()-idx] = self.dataset[idx:]
                return item, item

        else:
            if self.transforms is not None:
                return self.transforms(self.dataset[idx:idx+self.seq_len]), self.transforms(self.dataset[idx:idx+self.seq_len])
            else:
                return self.dataset[idx:idx+self.seq_len], self.dataset[idx:idx+self.seq_len]

                
# Helper for transforming the data from a list to Tensor
def listToTensor(list):
    tensor = torch.empty(list.__len__(), list[0].__len__())
    for i in range(list.__len__()):
        tensor[i, :] = torch.from_numpy(list[i])
    return tensor 

# transform
data_transform = transforms.Lambda(lambda x: listToTensor(x))  

In [226]:

# Dataset Objects
train_dataset = []
for df in df_train:
    data = np.array(df.iloc[:, 0].values).astype(float).reshape(-1, 1)
    train_dataset.append(SensorDataset(data, seq_len=data_seq_len, transform = data_transform))
val_dataset = []
for df in df_test:
    data = np.array(df.iloc[:401, 0].values).astype(float).reshape(-1, 1)
    val_dataset.append(SensorDataset(data, seq_len=data_seq_len, transform = data_transform))
private_dataset = []
for df in df_private:
    data = np.array(df.iloc[:, 0].values).astype(float).reshape(-1, 1)
    private_dataset.append(SensorDataset(data, seq_len=data_seq_len, transform = data_transform))

# Pytorch DataLoader objects
train_loader = []
for dataset in train_dataset:
    train_loader.append(DataLoader(dataset, batch_size=batch_size, shuffle=False))
val_loader = []
for dataset in val_dataset:
    val_loader.append(DataLoader(dataset, batch_size=batch_size, shuffle=False))
private_loader = []
for dataset in private_dataset:
    private_loader.append(DataLoader(dataset, batch_size=batch_size, shuffle=False))

In [227]:
class LSTMEncoder(nn.Module):
    
    def __init__(self, seq_len, n_features, embedding_dim):
        super(LSTMEncoder, self).__init__()
        
        # Parameters
        self.seq_len = seq_len
        self.n_features = n_features
        self.embedding_dim = embedding_dim
        self.hidden_dim = 2*embedding_dim
        
        # Neural Network Layers
        self.lstm1 = nn.LSTM(self.n_features, self.hidden_dim, num_layers=1, batch_first=True)
        self.lstm2 = nn.LSTM(self.hidden_dim, self.embedding_dim, num_layers=1, batch_first=True)
    
    def forward(self, i): 
        i, _ = self.lstm1(i)               # from (batch, seq_len, n_features) to (batch, seq_len, hidden_dim)
        i, (hidden_n, _) = self.lstm2(i)   # from (batch, seq_len, hidden_dim) to (batch, seq_len, embedding_dim)
        return hidden_n                    # hidden_n shape: (num_layers*num_directions, batch, embedding_dim)


class LSTMDecoder(nn.Module):

    def __init__(self, seq_len, embedding_dim, n_features=1):
        super(LSTMDecoder, self).__init__()

        # Parameters
        self.seq_len = seq_len
        self.embedding_dim = embedding_dim
        self.hidden_dim = 2*embedding_dim
        self.n_features = n_features
        
        # Neural Network Layers
        self.lstm1 = nn.LSTM(self.embedding_dim, self.embedding_dim, num_layers=1, batch_first=True)
        self.lstm2 = nn.LSTM(self.embedding_dim, self.hidden_dim, num_layers=1, batch_first=True)
        self.output_layer = nn.Linear(self.hidden_dim, n_features)
        
    def forward(self, i):
        # Do padding
        i = i.repeat(self.seq_len, 1, 1)                       # repeat (1, embedding_dim) to (seq_len, embedding_dim)
        i = i.reshape((-1, self.seq_len, self.embedding_dim))  # reshape to (batch, seq_len, embedding_dim)
        
        # Traverse neural layers
        i, _ = self.lstm1(i)      # from (batch, seq_len, embedding_dim) to (batch, seq_len, embedding_dim)
        i, _ = self.lstm2(i)      # from (batch, seq_len, embedding_dim) to (batch, seq_len, hidden_dim)
        i = self.output_layer(i)  # from (batch, seq_len, hidden_dim) to (batch, seq_len, n_features)
        
        return i


class LSTMAutoencoder(nn.Module):
    def __init__(self, seq_len, n_features, embedding_dim=64):
        super(LSTMAutoencoder, self).__init__()
        self.encoder = LSTMEncoder(seq_len, n_features, embedding_dim).to(device)
        self.decoder = LSTMDecoder(seq_len, embedding_dim, n_features).to(device)
        
    def forward(self, i):
        i = self.encoder(i)
        i = self.decoder(i)
        return i

In [228]:
# Define model1 for E1
model = LSTMAutoencoder(data_seq_len, data_n_features, data_embedding_dim)
model = model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
lr_schedule = cosine_scheduler(
        lr,
        min_lr,
        epochs, len(train_loader[0]),
        warmup_epochs=10,
    )

In [229]:
def training(model, optimizer, trainset_iterator, validationset_iterator, epoch):
    train_losses, test_losses = [], []
    criterion = nn.L1Loss(reduction='sum').to(device)

    for epoch in range(epoch):
        
        print("Epoch %d training started ..." % epoch)
        start_time = time.time()
        
        # Enter Train Mode
        model.train()     
        train_loss = 0
        for it, (ii, _) in enumerate(trainset_iterator):
            #print(ii)
            ii = ii.to(device)              # move to GPU if necessary
            it = len(trainset_iterator) * epoch + it  # global training iteration
            for i, param_group in enumerate(optimizer.param_groups):
                param_group["lr"] = lr_schedule[it]
            optimizer.zero_grad()           # generate prediction
            preds = model(ii)               # generate prediction
            loss = criterion(preds, ii)     # calculate loss
            loss.backward()                 # back propagation of gradients and update weights
            optimizer.step()                # update optimizer
            train_loss += loss.item()       # record training losses

        # Enter Validation Mode
        model.eval()
        test_loss = 0
        with torch.no_grad():
            for ii, _ in validationset_iterator:
                ii = ii.to(device)          # move to GPU if necessary
                preds = model(ii)           # generate prediction
                loss = criterion(preds, ii) # calculate loss
                test_loss += loss           # record validation testing losses
        
        end_time = time.time()
        train_losses.append(train_loss)
        test_losses.append(test_loss)
        print("Epoch %d completed - train_loss: %f , test_loss: %f" % (epoch, train_loss, test_loss))
        print("Epoch %d training time: %f" %(epoch, (end_time - start_time)))
    
    return train_losses, test_losses

In [270]:
def predict(model, dataset_iterator):
    preds, losses = np.array([]), np.array([])
    criterion = nn.L1Loss(reduction='none').to(device)
    
    # Enter Validation Mode
    model = model.eval()
    with torch.no_grad():
        for ii, _ in dataset_iterator:
            # move to GPU if necessary
            ii = ii.to(device)
            
            # generate prediction
            pred = model(ii)
            
            # calculate loss
            loss = criterion(pred, ii)
            
            # record predictions
            preds = np.append(preds, pred[:, -1, :].cpu().numpy())
            
            # record mean loss of each sample
            loss = loss.reshape((-1, data_seq_len)).cpu().numpy()  # from (batch, seq_len, n_feature) to (batch, seq_len)
            losses = np.append(losses, [np.sum(i) for i in loss])  # sum of all seq_len losses into one loss
    
    preds = preds.reshape((-1, dataset_iterator.__len__))  # reshape to (batch, seq_len)
    return preds, losses

In [231]:
for i in range(5):
    # Training:
    train_losses, test_losses = training(model, optimizer, train_loader[i], val_loader[i], epochs)

    # Saving trained models
    torch.save(model, f'./model/sensor_model_{i}.pth')

Epoch 0 training started ...
Epoch 0 completed - train_loss: 218496.298828 , test_loss: 279288.187500
Epoch 0 training time: 5.816305
Epoch 1 training started ...
Epoch 1 completed - train_loss: 215234.703125 , test_loss: 271115.531250
Epoch 1 training time: 5.726530
Epoch 2 training started ...
Epoch 2 completed - train_loss: 207613.662109 , test_loss: 255969.859375
Epoch 2 training time: 5.701616
Epoch 3 training started ...
Epoch 3 completed - train_loss: 193724.263672 , test_loss: 227306.171875
Epoch 3 training time: 5.711325
Epoch 4 training started ...
Epoch 4 completed - train_loss: 163961.347656 , test_loss: 152052.703125
Epoch 4 training time: 5.706374
Epoch 5 training started ...
Epoch 5 completed - train_loss: 121374.203125 , test_loss: 152850.359375
Epoch 5 training time: 5.699254
Epoch 6 training started ...
Epoch 6 completed - train_loss: 121784.672852 , test_loss: 168966.625000
Epoch 6 training time: 5.717803
Epoch 7 training started ...
Epoch 7 completed - train_loss: 1

# Evaluate result on public

In [263]:
# load val data
df_sensorA_public = pd.read_csv('data/sensor_A_public.csv')
df_sensorB_public = pd.read_csv('data/sensor_B_public.csv')
df_sensorC_public = pd.read_csv('data/sensor_C_public.csv')
df_sensorD_public = pd.read_csv('data/sensor_D_public.csv')
df_sensorE_public = pd.read_csv('data/sensor_E_public.csv')
print(len(df_sensorA_public))
df_test = [df_sensorA_public, df_sensorB_public, df_sensorC_public, df_sensorD_public, df_sensorE_public]
target = []
for df in df_test:
    target.append(df.iloc[:, -1].values)
    df.drop(df.index[-1], inplace=True)

4000


In [264]:
val_dataset = []
for df in df_test:
    data = np.array(df.iloc[:, 0].values).astype(float).reshape(-1, 1)
    val_dataset.append(SensorDataset(data, seq_len=data_seq_len, transform = data_transform))
val_loader = []
for dataset in val_dataset:
    val_loader.append(DataLoader(dataset, batch_size=batch_size, shuffle=False))

In [271]:
for i in range(5):
    model = torch.load(f'./model/sensor_model_{i}.pth')
    reconstructed_val, losses_val = predict(model, val_loader[i])
    print(reconstructed_val.shape)
    origin_data = val_dataset[i].dataset
    print(origin_data)
    reconstructed_error = np.abs((reconstructed_val - origin_data))
    print("reconstructed_error:", reconstructed_error.shape)
    print("AUC score:", roc_auc_score(target[i], reconstructed_error))

TypeError: 'method' object cannot be interpreted as an integer

# Generate private result